# README
**方言コーパスは個人利用にとどめることを強く推奨します。**
**本プログラムの商用利用はご遠慮ください。**
学術**論文**での利用を希望される場合、[Bandolu (Kogane)](https://github.com/Bandolu)までご連絡ください。

#このプログラムについて
方言コーパスを平仮名に変換します。
標準語の漢字入力も平等に平仮名に変えます。

方言を平仮名にした後漢字に変換し直す労力を割ける人は[関数定義](https://colab.research.google.com/drive/1XxFAiD2lK-kExoEK5VwQUkwCvWGaUX5N?authuser=2#scrollTo=jtEwt4a327y3&line=1&uniqifier=1)セルを上手くイジってください。  
Driveマウントの項はご自由にいじってどうぞ。

##CSVフォーマット
[ここ](https://www2.ninjal.ac.jp/cojads/index.html?targ=data)からコーパスをDLして、列を絞ってください。  
再配布はしたくないので、結合作業は自分で頑張ってください。Excelがあれば簡単にできます。  

|  話者  |  地名  |  方言テキスト  |  標準語テキスト  |
| ---- | ---- | ---- | ---- |
|  TD  |  TD  | TD  |  TD  |
|  TD  |  TD  | TD  |  TD  |


### 参考にさせていただきました
[Python] 結合文字を使用した濁点や半濁点を直前の仮名と結合させる方法(ウ゛→ ヴ) - Qiita https://qiita.com/gretchi/items/13c0825282415e2e360d


#Driveマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Transformer

# ライブラリ読み込み

In [ ]:
%%bash 
pip install  jaconv
pip install icecream
pip install pykakasi

In [ ]:
import pandas as pd
import re
import codecs
import copy
import jaconv
import os
from icecream import ic
import unicodedata
from pykakasi import kakasi

###デバッグ不要なら以下のセルを起動すべし。

In [ ]:
ic.disable()

# データ読み込み

In [ ]:
in_file = "0001_sendai_v1_utf8.csv"
corpus_path = os.path.join("DATA",in_file)
df = pd.read_csv(corpus_path)
#df['A'] = ""
ic('**** Amount of data ****')
ic(df)
ic('\n')
ic('**** Amount of data ****')


# 関数定義

In [ ]:
# オブジェクトをインスタンス化
kakasi = kakasi()

In [ ]:
#c_path = './DATA/Kessen.txt'

c_df = copy.deepcopy(df)
pHogen = re.compile(r'[（ＦF：）ＤＺＭ〔〕＊［］Ｏ１ＳＧＣＰＬＲX123SP＜＞＝｛｝「」]')
pBigakuon = re.compile(r'[（ＦF：）ＤＺＭ〔〕＊［］Ｏ１ＳＧＣＰＬＲX123SP＜＞＝｛｝「」]')


def kata_hira(x):
    return jaconv.kata2hira(x)

def regex(x):
    return re.sub(pHogen, '', x)


def noma(x):
    return unicodedata.normalize('NFKC', x)

def join_diacritic(_text):
    """
    基底文字と濁点・半濁点を結合
    """

    # 謎の○を濁点に戻す
    text = re.sub('゜', '゛', _text)
    # str -> bytes
    bytes_text = text.encode()



    # 濁点Unicode結合文字置換
    bytes_text = re.sub(b"\xe3\x82\x9b", b'\xe3\x82\x99', bytes_text)
    bytes_text = re.sub(b"\xef\xbe\x9e", b'\xe3\x82\x99', bytes_text)

    # 半濁点Unicode結合文字置換
    bytes_text = re.sub(b"\xe3\x82\x9c", b'\xe3\x82\x9a', bytes_text)
    bytes_text = re.sub(b"\xef\xbe\x9f", b'\xe3\x82\x9a', bytes_text)

    # bytet -> str
    text = bytes_text.decode()

    # 正規化
    mode="NFKC"
    text = unicodedata.normalize(mode, text)

    return text


def kanji2hiragana(x):
    text = x

    # モードの設定：J(Kanji) to H(Hiragana)
    kakasi.setMode('J', 'H') 

    # 変換して出力
    conv = kakasi.getConverter()
    text = conv.do(text)  # => けいたいそかいせき
    return text


#実行

In [ ]:

hougen = c_df['方言テキスト']
hyoujun = c_df['標準語テキスト']
ic(hougen)
ic(hyoujun)

hougen = hougen.map(regex)
hougen = hougen.map(join_diacritic)
hougen = hougen.map(kata_hira)
#ic(hougen)
hyoujun = hyoujun.map(regex)
hyoujun = hyoujun.map(kanji2hiragana)
hyoujun = hyoujun.map(kata_hira)
hyoujun = hyoujun.map(noma)
#ic(hyoujun)
        
        #print(parsed,file=f)
ic(c_df.head())

c_df['方言テキスト'] = hougen
c_df['標準語テキスト'] = hyoujun
ic(c_df.head())

# データ出力

In [ ]:
c_df.to_csv( os.path.join("DATA","Hiragana_"+in_file), index=False)